In [1]:
!hf cache scan

REPO ID                          REPO TYPE SIZE ON DISK NB FILES LAST_ACCESSED LAST_MODIFIED REFS LOCAL PATH                                                                   
-------------------------------- --------- ------------ -------- ------------- ------------- ---- ---------------------------------------------------------------------------- 
meta-llama/Llama-3.2-3B-Instruct model            12.9G       15 3 hours ago   3 hours ago   main /home/jhkim/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct 
mistralai/Voxtral-Mini-3B-2507   model             9.4G        7 4 minutes ago 2 weeks ago   main /home/jhkim/.cache/huggingface/hub/models--mistralai--Voxtral-Mini-3B-2507   

Done in 0.0s. Scanned 2 repo(s) for a total of 22.2G.
Got 2 warning(s) while scanning. Use -vvv to print details.


In [ ]:
import transformers
import torch
import os

model_id = os.getenv("MODEL_PATH", "meta-llama/Llama-3.2-3B-Instruct")

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "당신은 의사 입니다, 환자의 질문에 한국어로 답하세요"},
    {"role": "user", "content": "성병이 성적 접촉을 통해 전파되는 질병인지, 그리고 치료를 받지 않으면 심각한 합병증을 유발할 수 있는가?"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'role': 'assistant', 'content': '성병은 성적 접촉을 통해 전파되는 질병이지만, 모든 성병은 치료를 받지 않으면 심각한 합병증을 유발하지 않는 것은 아닙니다.\n\n성병은 HIV(인플루エ나)와 같은 유전적 질병을 포함한 다양한类型의 질병이 있습니다. HIV는 성적 접촉을 통해 전파되는 질병으로, 치료를 받지 않으면 AIDS(인플루エ나 에이즈)로 진화할 수 있습니다. AIDS는 심각한 합병증을 유발할 수 있습니다, chẳng hạn như 감염, 감염, 심장 질환, 그리고 심지어 사망.\n\n다른 성병과는 다르게, HIV는 치료를 받지 않으면 심각한 합병증을 유발하지 않는 것은 아닙니다. HIV는 치료를 받고 제어할 수 있으며, 치료를 받은 사람들은 일반적으로 AIDS로 진화하지 않습니다.\n\n그러나, 성병은 심각한 합병증을 유발할 수 있습니다. 예를 들어, 성병은 감염, 감염, 심장 질환, 그리고 심지어 사망'}


In [ ]:
"""
간단한 Voxtral 오디오 추론 코드

이 스크립트는 오디오 파일을 입력받아 Voxtral 모델로 추론을 수행합니다.
sample.py에서 핵심 기능만 추출하여 단순화한 버전입니다.

# audio + text
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "audio", "url": audio_url},
            {"type": "audio", "path": audio_path},
            {"type": "audio", "base64": audio_base64},
            {"type": "text", "text": "How many audio do you hear?"},
        ],
    },
]

processor = VoxtralProcessor.from_pretrained("mistralai/Voxtral-Mini-3B-2507")
inputs = processor.apply_chat_template(conversation)

# transcription
inputs = processor.apply_transcription_request(language=language, audio=audio, model_id=model_id)

"""

import argparse
import torch
from transformers import VoxtralForConditionalGeneration, AutoProcessor
import librosa

device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = os.getenv("MODEL_PATH", "/shared/models/mistralai/Voxtral-Mini-3B-2507")

print(f"사용 디바이스: {device}")
print(f"모델 로딩 중: {model_id}")

processor = AutoProcessor.from_pretrained(model_id)
model = VoxtralForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    device_map=device
)

audio_path = "/shared/home/milab/users/kskim/dataset/138.뉴스_대본_및_앵커_음성_데이터/01-1.정식개방데이터/Validation/01.원천데이터/SPK003/SPK003YTNSO162/SPK003YTNSO162F001.wav"
# audio = librosa.load(audio_path, sr=16000)[0]

# inputs = processor.apply_transcription_request(language="en", audio=audio_path, model_id=model_id)

# inputs = inputs.to(device, dtype=torch.bfloat16 if device == "cuda" else torch.float32)
# outputs = model.generate(**inputs, max_new_tokens=500)
# result = processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)


# print(result[0])


# audio_path = '/home/DB/jhkim/speechllm/adapter_finetuning/sample_audio/SPK009YTNSO123M007.wav'
text = 'ㅇㅣㅁ'

# audio + text
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "audio", "path": audio_path},
            # {"type": "text", "text": text},
        ],
    },
]

inputs = processor.apply_chat_template(conversation)
inputs = inputs.to(device, dtype=torch.bfloat16 if device == "cuda" else torch.float32)
outputs = model.generate(**inputs, max_new_tokens=500)
result = processor.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)
print(result[0])